# Capstone Project - The Battle of Neighborhoods

## Clustering the Neighborhoods of Amsterdam, The Netherlands

##### Use the Foursquare API to explore  the neighborhoods of Amsterdam and obtain the most common venue categories in each neighborhood. Cluster the neighborhoods using the k-means algorithm. Finally use the Folium library to visualize these clusters superimposed onto the map of Amsterdam. 

##### Obtain the geodata of the neighborhoods of Amsterdam from https://maps.amsterdam.nl/open_geodata/?LANG=en. As there are 481 neighborhoods in Amsterdam use the 99 aggregated neighborhoods, called quarters. Use the data excluding the major waters. For simplicity refer to these quarters as neighborhoods. 

Import all the libraries that will be needed.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np
import requests

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

#!conda install -c conda-forge folium=0.5.0 --yes
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

### Import and Explore Dataset

The data is available in different formats. Note: the separator in the **csv** file  is not a comma (,) but a semi-colon (;).  

In [2]:
amsterdam_data=pd.read_csv('GEBIED_BUURTCOMBINATIES_EXWATER.csv',sep=';')
print('There are {} neighborhoods in Amsterdam.'.format(amsterdam_data.shape[0]))
amsterdam_data.head(2)

There are 99 neighborhoods in Amsterdam.


,OBJECTNUMMER,Buurtcombinatie_code,Buurtcombinatie,Stadsdeel_code,Opp_m2,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT,Unnamed: 9
0,1,T92,Amstel III/Bullewijk,T,5273610,"POLYGON((4.971842 52.284355,4.970005 52.286362...","POLYGON((52.284355 4.971842,52.286362 4.970005...","4,9505918","52,298741",NaN
1,2,T93,"Bijlmer Centrum (D,F,H)",T,3158100,"POLYGON((4.971421 52.307989,4.965179 52.314811...","POLYGON((52.307989 4.971421,52.314811 4.965179...","4,9545269","52,3153044",NaN


Remove the columns that are not needed. If applicable, translate the remaining column names from Dutch to English. 

In [3]:
amsterdam_data=amsterdam_data.drop(columns=['OBJECTNUMMER','Buurtcombinatie_code','Stadsdeel_code','Opp_m2','WKT_LNG_LAT','WKT_LAT_LNG','Unnamed: 9'])
amsterdam_data = amsterdam_data.rename(columns={'Buurtcombinatie': 'Neighborhood'})
amsterdam_data.head()

,Neighborhood,LNG,LAT
0,Amstel III/Bullewijk,"4,9505918","52,298741"
1,"Bijlmer Centrum (D,F,H)","4,9545269","52,3153044"
2,Gein,"4,99393105","52,29550995"
3,Kinkerbuurt,"4,86544515","52,36808265"
4,Overtoomse Sluis,"4,8598361","52,3597247"


In [4]:
amsterdam_data.dtypes

Neighborhood    object
LNG             object
LAT             object
dtype: object

Fix the  problems with the latitude (LAT) and longitude (LNG) columns: change the decimal separator from comma to dot and change the data type to float.

In [5]:
amsterdam_data["LAT"] = amsterdam_data["LAT"].str.replace(",",".").astype(float)
amsterdam_data["LNG"] = amsterdam_data["LNG"].str.replace(",",".").astype(float)

amsterdam_data.dtypes

Neighborhood     object
LNG             float64
LAT             float64
dtype: object

#### Visualize the neighborhoods 

Use the  **geopy** library to get the latitude and longitude values of Amsterdam.

In [6]:
address = 'Amsterdam'

geolocator = Nominatim(user_agent="ams_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Amsterdam are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Amsterdam are 52.3745403, 4.89797550561798.


Visualize the neighborhoods superimposed on the map of Amsterdam.

In [7]:
# create map of Amsterdam using latitude and longitude values
map_amsterdam = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(amsterdam_data['LAT'], amsterdam_data['LNG'], amsterdam_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_amsterdam)  
    
map_amsterdam

### Define Foursquare Credentials and Version

In [8]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Define the function **getNearbyVenues** which will be used to explore the venues in the neighborhoods of Amsterdam  

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
       
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Call the above defined function to explore all the neighborhoods of Amsterdam.

In [10]:
LIMIT=100
amsterdam_venues = getNearbyVenues(names=amsterdam_data['Neighborhood'],
                                   latitudes=amsterdam_data['LAT'],
                                   longitudes=amsterdam_data['LNG']
                                  )


Amstel III/Bullewijk
Bijlmer Centrum (D,F,H)
Gein
Kinkerbuurt
Overtoomse Sluis
Erasmuspark
Hoofdweg e.o.
De Weteringschans
Oostelijke Eilanden/Kadijken
Volewijck
Tuindorp Nieuwendam
Tuindorp Buiksloot
Kadoelen
Elzenhagen
Burgwallen-Oude Zijde
Bijlmer Oost (E,G,K)
Driemond
Haarlemmerbuurt
Osdorp-Oost
De Punt
Westelijk Havengebied
Centrale Markt
Westlandgracht
Zuidas
Slotermeer-Noordoost
Geuzenveld
IJburg West
Betondorp
Transvaalbuurt
Indische Buurt Oost
Nieuwe Pijp
Rijnbuurt
Da Costabuurt
Helmersbuurt
Staatsliedenbuurt
Geuzenbuurt
Sloterdijk
Burgwallen-Nieuwe Zijde
Westindische Buurt
Houthavens
Eendracht
Slotermeer-Zuidwest
Osdorp-Midden
Slotervaart Zuid
Oude Pijp
Schinkelbuurt
Apollobuurt
IJselbuurt
Buitenveldert-West
Oosterparkbuurt
Indische Buurt West
Zeeburgereiland/Nieuwe Diep
Nieuwendammerdijk/Buiksloterdijk
Tuindorp Oostzaan
Nellestein
Frederik Hendrikbuurt
Grachtengordel-West
Spaarndammer- en Zeeheldenbuurt
Van Lennepbuurt
Nieuwmarkt/Lastage
De Kolenkit
Slotervaart Noord
Lutkeme

In [11]:
print('Total venues found:', amsterdam_venues.shape[0])
amsterdam_venues.head()

Total venues found: 6533


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Amstel III/Bullewijk,52.298741,4.950592,Hotel Lowell,52.304526,4.950671,Hotel
1,Amstel III/Bullewijk,52.298741,4.950592,De Proefzaak,52.303585,4.950890,Café
2,Amstel III/Bullewijk,52.298741,4.950592,Fitness365,52.298421,4.945692,Gym
3,Amstel III/Bullewijk,52.298741,4.950592,Museum Vrolik,52.293968,4.960014,Science Museum
4,Amstel III/Bullewijk,52.298741,4.950592,Joy,52.305268,4.950310,Hotel


List the number of venues found for each neighborhood.

In [12]:
amsterdam_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Amstel III/Bullewijk,43,43,43,43,43,43
Apollobuurt,100,100,100,100,100,100
Banne Buiksloot,19,19,19,19,19,19
Bedrijventerrein Sloterdijk,9,9,9,9,9,9
Betondorp,30,30,30,30,30,30
"Bijlmer Centrum (D,F,H)",93,93,93,93,93,93
"Bijlmer Oost (E,G,K)",14,14,14,14,14,14
Buikslotermeer,33,33,33,33,33,33
Buitenveldert-Oost,82,82,82,82,82,82


Remove all neighborhoods that have less then 10 venues found in them. 

In [18]:
df=amsterdam_venues.groupby('Neighborhood').count()<10
df=df[df['Venue']==True]
neigh=df.index.values
neigh

array(['Bedrijventerrein Sloterdijk', 'Waterland',
       'Westelijk Havengebied'], dtype=object)

In [32]:
for n in neigh:
    print(n)
    amsterdam_venues=amsterdam_venues.drop(amsterdam_venues[amsterdam_venues['Neighborhood']== n].index)

amsterdam_venues.groupby('Neighborhood').count()


Bedrijventerrein Sloterdijk
Waterland
Westelijk Havengebied


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Amstel III/Bullewijk,43,43,43,43,43,43
Apollobuurt,100,100,100,100,100,100
Banne Buiksloot,19,19,19,19,19,19
Betondorp,30,30,30,30,30,30
"Bijlmer Centrum (D,F,H)",93,93,93,93,93,93
"Bijlmer Oost (E,G,K)",14,14,14,14,14,14
Buikslotermeer,33,33,33,33,33,33
Buitenveldert-Oost,82,82,82,82,82,82
Buitenveldert-West,24,24,24,24,24,24


In [33]:
print('There are {} uniques categories.'.format(len(amsterdam_venues['Venue Category'].unique())))

There are 311 uniques categories.


Transform the amsterdam_venues dataframe to make it suitable for the cluster analysis. The dataframe is expended with as many columns as unique venue categories.   

In [34]:
# one hot encoding
amsterdam_onehot = pd.get_dummies(amsterdam_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
amsterdam_onehot['Neighborhood'] = amsterdam_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [amsterdam_onehot.columns[-1]] + list(amsterdam_onehot.columns[:-1])
amsterdam_onehot = amsterdam_onehot[fixed_columns]

amsterdam_onehot.head()

,Zoo Exhibit,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brasserie,Breakfast Spot,Brewery,Bridal Shop,Bridge,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Cigkofte Place,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cruise,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Doner Restaurant,Drugstore,Dumpling Restaurant,Dutch Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Friterie,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Field,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,Nightclub,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Piano Bar,Pier,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Swiss

Group the dataframe by neighborhood

In [35]:
amsterdam_grouped = amsterdam_onehot.groupby('Neighborhood').mean().reset_index()
amsterdam_grouped.head()

,Neighborhood,Zoo Exhibit,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brasserie,Breakfast Spot,Brewery,Bridal Shop,Bridge,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Cigkofte Place,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cruise,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Doner Restaurant,Drugstore,Dumpling Restaurant,Dutch Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Friterie,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Field,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Nature Preserve,Nightclub,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Piano Bar,Pier,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Swiss

##### Before running the clustering algorithm display the 10 most common venues in each neighborhood.

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = amsterdam_grouped['Neighborhood']

for ind in np.arange(amsterdam_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(amsterdam_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amstel III/Bullewijk,Hotel,Restaurant,Fast Food Restaurant,Coffee Shop,Food Court,Café,Furniture / Home Store,Shopping Mall,Lounge,Food & Drink Shop
1,Apollobuurt,Hotel,Restaurant,Bakery,Coffee Shop,French Restaurant,Italian Restaurant,Plaza,Japanese Restaurant,Bar,Bistro
2,Banne Buiksloot,Soccer Field,Bus Stop,Park,Sports Club,Church,Grocery Store,Trail,Gym / Fitness Center,Monument / Landmark,Turkish Restaurant
3,Betondorp,Soccer Field,Café,Stadium,Plaza,Scandinavian Restaurant,Sports Club,Sports Bar,Flower Shop,Museum,Music Venue
4,"Bijlmer Centrum (D,F,H)",Office,Chinese Restaurant,Park,Coffee Shop,Bakery,Restaurant,Plaza,Food Court,Hotel,Hostel
5,"Bijlmer Oost (E,G,K)",Supermarket,Bus Stop,South American Restaurant,Turkish Restaurant,Bar,Discount Store,Park,Shopping Mall,Drugstore,Metro Station
6,Buikslotermeer,Supermarket,Bus Stop,Clothing Store,Park,Department Store,Mobile Phone Shop,Seafood Restaurant,Electronics Store,Soccer Field,Snack Place
7,Buitenveldert-Oost,Hotel,Gym,Korean Restaurant,Gym / Fitness Center,Bakery,Asian Restaurant,Supermarket,Playground,Platform,Restaurant
8,Buitenveldert-West,Harbor / Marina,Tennis Court,Restaurant,Pool,Trail,Botanical Garden,Boat or Ferry,Spa,Stables,Café
9,Burgwallen-Nieuwe Zijde,Hotel,Café,Marijuana Dispensary,Bar,French Restaurant,Cocktail Bar,Sandwich Place,Pub,Cheese Shop,Chinese Restaurant


In [59]:
# set number of clusters
kclusters = 20

#remove the column containing the name of the Neighborhood
amsterdam_grouped_clustering = amsterdam_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(amsterdam_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([ 0, 11,  2,  7,  0,  3,  6,  0, 13, 11, 11,  1,  1,  1,  1,  0, 17,
        8,  5,  4,  6,  0,  0,  1, 10,  1,  9, 11,  8, 11,  1, 18, 11,  1,
       11,  0, 13,  0,  0,  8,  0, 11, 11,  2,  1,  0, 15, 19,  7, 11, 12,
        8,  6, 11, 11, 11,  0,  0, 11, 11, 14, 17, 17,  8, 11,  0,  8,  0,
        8, 11,  0,  0, 17, 17,  0,  0, 11,  1, 11, 11,  6,  9, 16,  0,  1,
        0, 11,  9, 11,  8, 11,  0, 11, 13,  8,  0], dtype=int32)

Insert the cluster label column into the dataframe and the geocoordinates

In [60]:
neighborhoods_venues_sorted.drop(columns=['Cluster Labels'],inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

amsterdam_merged = amsterdam_data
# merge amsterdam_grouped with amsterdam_data to add latitude/longitude for each neighborhood
amsterdam_merged = amsterdam_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how='right')

amsterdam_merged.shape

(96, 14)

##### Visualize the map of Amsterdam with the neighborhoods superimposed, colorcoded as a function of the cluster they belong to.

In [62]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.tab20(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(amsterdam_merged['LAT'], amsterdam_merged['LNG'], amsterdam_merged['Neighborhood'], amsterdam_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Conclusions

##### Display the top 10 venues for each neighborhood for each cluster.

In [42]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 0, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Bijlmer Centrum (D,F,H)",Office,Chinese Restaurant,Park,Coffee Shop,Bakery,Restaurant,Plaza,Food Court,Hotel,Hostel
18,Osdorp-Oost,Supermarket,Turkish Restaurant,Snack Place,Clothing Store,Bus Stop,Hotel,Mediterranean Restaurant,Chinese Restaurant,Tram Station,Shopping Mall
19,De Punt,Supermarket,Fast Food Restaurant,Clothing Store,Hotel,Turkish Restaurant,Bakery,Electronics Store,Steakhouse,Lounge,Sporting Goods Shop
22,Westlandgracht,Supermarket,Soccer Field,Hotel,Café,Restaurant,Coffee Shop,Gym,Tram Station,Train Station,Gym / Fitness Center
24,Slotermeer-Noordoost,Hotel,Turkish Restaurant,Park,Dessert Shop,Hostel,Department Store,Fish Market,Metro Station,Sandwich Place,Bed & Breakfast
26,IJburg West,Tram Station,Italian Restaurant,Park,Hotel,Greek Restaurant,Fish Market,Shopping Mall,Farmers Market,Soccer Field,Food
41,Slotermeer-Zuidwest,Turkish Restaurant,Tram Station,Park,Supermarket,Discount Store,Restaurant,Dessert Shop,Bus Stop,Plaza,Sporting Goods Shop
42,Osdorp-Midden,Supermarket,Tram Station,Fast Food Restaurant,Snack Place,Turkish Restaurant,Bakery,Drugstore,Mediterranean Restaurant,Hotel,Shopping Mall
43,Slotervaart Zuid,Tram Station,Supermarket,Plaza,Restaurant,Flower Shop,Asian Restaurant,Bus Stop,Train Station,Coffee Shop,Chinese Restaurant
51,Zeeburgereiland/Nieuwe Diep,Harbor / Marina,Hockey Field,Burger Joint,Beach,Supermarket,Soccer Field,Fish Market,Fast Food Restaurant,Farmers Market,Shopping Mall


In [43]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 1, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amstel III/Bullewijk,Hotel,Restaurant,Fast Food Restaurant,Coffee Shop,Food Court,Café,Furniture / Home Store,Shopping Mall,Lounge,Food & Drink Shop
5,Erasmuspark,Supermarket,Restaurant,Coffee Shop,Café,Bar,Bakery,Fast Food Restaurant,Park,Pizza Place,Bagel Shop
8,Oostelijke Eilanden/Kadijken,Zoo Exhibit,Restaurant,Bar,Café,Museum,Hotel,Gastropub,Breakfast Spot,Pizza Place,Performing Arts Venue
9,Volewijck,Café,Furniture / Home Store,Park,Bus Stop,Hotel,Restaurant,Grocery Store,Sporting Goods Shop,Snack Place,Bakery
23,Zuidas,Coffee Shop,Hotel,Restaurant,Supermarket,Bakery,Italian Restaurant,Breakfast Spot,Salad Place,Gym,Department Store
29,Indische Buurt Oost,Café,Restaurant,Coffee Shop,Plaza,Supermarket,Bakery,Harbor / Marina,Bar,Cocktail Bar,Convenience Store
31,Rijnbuurt,Café,Italian Restaurant,Supermarket,Restaurant,Gym / Fitness Center,Coffee Shop,Japanese Restaurant,Hotel,Sandwich Place,Park
36,Sloterdijk,Hotel,Restaurant,Music Venue,Farm,Supermarket,Bistro,Café,IT Services,Bar,Sushi Restaurant
39,Houthavens,Restaurant,Coffee Shop,Indie Movie Theater,Pizza Place,Café,Italian Restaurant,Bakery,Bar,Hotel,French Restaurant
46,Apollobuurt,Hotel,Restaurant,Bakery,Coffee Shop,French Restaurant,Italian Restaurant,Plaza,Japanese Restaurant,Bar,Bistro


In [44]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 2, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Geuzenveld,Bus Stop,Plaza,Supermarket,Tram Station,Grocery Store,Park,Track,Snack Place,Soccer Field,Market


In [45]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 3, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Tuindorp Nieuwendam,Bus Stop,Supermarket,Discount Store,Harbor / Marina,Snack Place,Bakery,Chinese Restaurant,Grocery Store,Park,Bar
11,Tuindorp Buiksloot,Park,Clothing Store,Bus Stop,Supermarket,Hotel,Seafood Restaurant,Skate Park,Harbor / Marina,Market,Turkish Restaurant
13,Elzenhagen,Supermarket,Bus Stop,Clothing Store,Department Store,Soccer Field,Pool,Church,Chinese Restaurant,Discount Store,Restaurant
15,"Bijlmer Oost (E,G,K)",Supermarket,Bus Stop,South American Restaurant,Turkish Restaurant,Bar,Discount Store,Park,Shopping Mall,Drugstore,Metro Station
52,Nieuwendammerdijk/Buiksloterdijk,Clothing Store,Bus Stop,Supermarket,Bakery,Park,Department Store,Seafood Restaurant,Market,Boat or Ferry,Soccer Field
53,Tuindorp Oostzaan,Supermarket,Furniture / Home Store,Theater,Sandwich Place,Restaurant,Department Store,Baseball Field,Flea Market,Bus Stop,Chinese Restaurant
67,Oostzanerwerf,Soccer Field,Supermarket,Hotel,Park,Bus Stop,Bakery,Pier,Flower Shop,Creperie,BBQ Joint
69,Waterlandpleinbuurt,Bakery,Grocery Store,Bus Stop,Athletics & Sports,Snack Place,Plaza,Pool,Stadium,Discount Store,Diner
80,Buikslotermeer,Supermarket,Bus Stop,Clothing Store,Park,Department Store,Mobile Phone Shop,Seafood Restaurant,Electronics Store,Soccer Field,Snack Place


In [46]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 4, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Betondorp,Soccer Field,Café,Stadium,Plaza,Scandinavian Restaurant,Sports Club,Sports Bar,Flower Shop,Museum,Music Venue
40,Eendracht,Soccer Field,Garden,General Entertainment,Event Space,Athletics & Sports,Recreation Center,Stadium,Cafeteria,Exhibit,Falafel Restaurant
92,Middenmeer,Soccer Field,Café,Coffee Shop,Playground,Fast Food Restaurant,Bus Stop,Restaurant,Train Station,Skating Rink,Chinese Restaurant


In [47]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 5, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Gein,Lake,Playground,Gym,Bus Stop,Smoke Shop,Metro Station,Supermarket,Drugstore,Dumpling Restaurant,Dutch Restaurant


In [48]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 6, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Kadoelen,Bus Stop,Soccer Field,Boat or Ferry,Gym / Fitness Center,Ski Area,Shopping Mall,Sports Club,Gas Station,Park,Trail
70,Banne Buiksloot,Soccer Field,Bus Stop,Park,Sports Club,Church,Grocery Store,Trail,Gym / Fitness Center,Monument / Landmark,Turkish Restaurant


In [66]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 7, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Gein,Lake,Playground,Metro Station,Bus Stop,Shopping Mall,Gym,Supermarket,Smoke Shop,Falafel Restaurant,Exhibit
54,Nellestein,Lake,Restaurant,Playground,Convention Center,Monument / Landmark,Hotel,Hotel Bar,Park,Stables,Campground


In [67]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 8, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Westelijk Havengebied,Harbor / Marina,Bar,Restaurant,Heliport,Event Space,Dumpling Restaurant,Dutch Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant


In [68]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 9, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Betondorp,Soccer Field,Café,Flower Shop,Stadium,Plaza,Chinese Restaurant,Fast Food Restaurant,Museum,Scandinavian Restaurant,Sporting Goods Shop
40,Eendracht,Soccer Field,Garden,General Entertainment,Cafeteria,Athletics & Sports,Stadium,Event Space,Zoo,Ethiopian Restaurant,Eastern European Restaurant
92,Middenmeer,Café,Soccer Field,Fast Food Restaurant,Restaurant,Train Station,Coffee Shop,Athletics & Sports,Supermarket,Furniture / Home Store,Bus Stop


In [64]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 10, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Gein,Lake,Playground,Gym,Bus Stop,Smoke Shop,Metro Station,Supermarket,Drugstore,Dumpling Restaurant,Dutch Restaurant


In [65]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 11, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Overtoomse Sluis,Restaurant,Bar,Café,Ethiopian Restaurant,Coffee Shop,Bistro,Music Venue,Yoga Studio,Bed & Breakfast,Breakfast Spot
8,Oostelijke Eilanden/Kadijken,Zoo Exhibit,Restaurant,Bar,Café,Museum,Hotel,Gastropub,Breakfast Spot,Pizza Place,Performing Arts Venue
14,Burgwallen-Oude Zijde,Hotel,Bar,French Restaurant,Marijuana Dispensary,Cocktail Bar,Coffee Shop,Chinese Restaurant,Bakery,Thai Restaurant,Cheese Shop
17,Haarlemmerbuurt,Bar,Restaurant,Marijuana Dispensary,Italian Restaurant,Café,Coffee Shop,Bistro,Bakery,Sandwich Place,Deli / Bodega
28,Transvaalbuurt,Café,French Restaurant,Coffee Shop,Hotel,Restaurant,Italian Restaurant,Bar,Turkish Restaurant,Breakfast Spot,Vegetarian / Vegan Restaurant
37,Burgwallen-Nieuwe Zijde,Hotel,Café,Marijuana Dispensary,Bar,French Restaurant,Cocktail Bar,Sandwich Place,Pub,Cheese Shop,Chinese Restaurant
38,Westindische Buurt,Restaurant,Bar,Vegetarian / Vegan Restaurant,Ethiopian Restaurant,Café,Music Venue,Coffee Shop,Bistro,Park,Breakfast Spot
39,Houthavens,Restaurant,Coffee Shop,Indie Movie Theater,Pizza Place,Café,Italian Restaurant,Bakery,Bar,Hotel,French Restaurant
45,Schinkelbuurt,Restaurant,Bar,Coffee Shop,Café,Gym / Fitness Center,Hotel,Bakery,Athletics & Sports,Vegetarian / Vegan Restaurant,Pizza Place
46,Apollobuurt,Hotel,Restaurant,Bakery,Coffee Shop,French Restaurant,Italian Restaurant,Plaza,Japanese Restaurant,Bar,Bistro


In [66]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 12, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,Nellestein,Lake,Restaurant,Playground,Snack Place,Park,Hotel,Hotel Bar,Convention Center,Campground,Monument / Landmark


In [67]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 13, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,IJburg West,Tram Station,Italian Restaurant,Park,Hotel,Greek Restaurant,Fish Market,Shopping Mall,Farmers Market,Soccer Field,Food
48,Buitenveldert-West,Harbor / Marina,Tennis Court,Restaurant,Pool,Trail,Botanical Garden,Boat or Ferry,Spa,Stables,Café
51,Zeeburgereiland/Nieuwe Diep,Harbor / Marina,Hockey Field,Burger Joint,Beach,Supermarket,Soccer Field,Fish Market,Fast Food Restaurant,Farmers Market,Shopping Mall


In [68]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 14, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
67,Oostzanerwerf,Soccer Field,Supermarket,Hotel,Park,Bus Stop,Bakery,Pier,Flower Shop,Creperie,BBQ Joint


In [69]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 15, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Lutkemeer/Ookmeer,Clothing Store,Discount Store,Turkish Restaurant,Café,Fast Food Restaurant,Electronics Store,Sandwich Place,Pet Store,Sporting Goods Shop,Piano Bar


In [70]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 16, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,Tuindorp Oostzaan,Supermarket,Furniture / Home Store,Theater,Sandwich Place,Restaurant,Department Store,Baseball Field,Flea Market,Bus Stop,Chinese Restaurant


In [71]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 17, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Osdorp-Oost,Supermarket,Turkish Restaurant,Snack Place,Clothing Store,Bus Stop,Hotel,Mediterranean Restaurant,Chinese Restaurant,Tram Station,Shopping Mall
19,De Punt,Supermarket,Fast Food Restaurant,Clothing Store,Hotel,Turkish Restaurant,Bakery,Electronics Store,Steakhouse,Lounge,Sporting Goods Shop
24,Slotermeer-Noordoost,Hotel,Turkish Restaurant,Park,Dessert Shop,Hostel,Department Store,Fish Market,Metro Station,Sandwich Place,Bed & Breakfast
41,Slotermeer-Zuidwest,Turkish Restaurant,Tram Station,Park,Supermarket,Discount Store,Restaurant,Dessert Shop,Bus Stop,Plaza,Sporting Goods Shop
42,Osdorp-Midden,Supermarket,Tram Station,Fast Food Restaurant,Snack Place,Turkish Restaurant,Bakery,Drugstore,Mediterranean Restaurant,Hotel,Shopping Mall


In [72]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 18, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
72,Holendrecht/Reigersbos,Pizza Place,Chinese Restaurant,Shopping Mall,Metro Station,Supermarket,Convenience Store,Platform,Pharmacy,Coffee Shop,Park


In [73]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 19, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
74,Middelveldsche Akerploder,Fast Food Restaurant,Pool,Auto Workshop,Harbor / Marina,Gym / Fitness Center,Gym,Grocery Store,Garden Center,Music Store,Deli / Bodega
